In [1]:
# combine eia dfs into one
# reword eia df location lists
# scrape noted areas during outages + 1 month before & after
# nlp tweets, model outage probability
# plot outage-related tweets w/ probabilities

## Read in Data

In [46]:
import numpy as np
import pandas as pd

import datetime

from twitterscraper import query_tweets

In [3]:
# list of dfs of eia data
xlsx_list = [pd.read_excel(f'{year}.xlsx', header=1) 
              for year in range(2013,2020)]

In [4]:
# combine dfs of eia data
df = pd.concat(xlsx_list, sort=False)

In [5]:
# function to determine if event is in washington
def washington_check(loc):
    right_wa = 'Washington'
    wrong_wa = ('D.C', 'Columbia', 'DC', 
        'Washington County', 'Oregon Counties')
    if right_wa in loc and not any(wrong in loc for wrong in wrong_wa):
        return True
    return False

In [6]:
# washington df
wa = df[df['Area Affected'].fillna('?').map(washington_check)]

In [7]:
# look at where in washington
wa['Area Affected'].value_counts()

Washington                                                                                                                                                                         8
Washington: Clark County;                                                                                                                                                          6
Washington:                                                                                                                                                                        4
Washington: King County;                                                                                                                                                           2
Washington: Snohomish County;                                                                                                                                                      2
Washington: King County, Whatcom County, Kitsap County, Skagit County;                         

### Create preliminary dataset based on most frequent county

In [21]:
# find most frequent county

allstring = ''
for s in wa['Area Affected']:
    allstring += s.replace(',','').replace(':','').replace(';','') + ' '
alllist = allstring.split()
alldict = {k:0 for k in set(alllist)}
for s in alllist:
    alldict[s] += 1
alldict

# king county is the most frequent county for outages in wa

{'Thurston': 6,
 'Kitsap': 7,
 'Island': 4,
 'Idaho': 2,
 'Montana': 2,
 'Stevens': 1,
 'Kittitas': 1,
 'Whatcom': 7,
 'counties': 1,
 'Whitman': 1,
 'Clark': 6,
 'Bonner': 1,
 'Alberta': 1,
 'State': 1,
 'Shoshone': 1,
 'Lincoln': 1,
 'Counties': 1,
 'Adams': 1,
 'Canada': 1,
 'County': 55,
 'Washington': 36,
 'Skagit': 6,
 'Spokane': 1,
 'King': 11,
 'Snohomish': 3,
 'and': 3,
 'Benewah': 1,
 'Pierce': 3,
 'Kootenai': 1}

In [63]:
# pick an outage date for king county
wa['Number of Customers Affected'] = wa['Number of Customers Affected'].map(lambda s: np.nan if (s == 'Unknown') else s)
wa[wa['Area Affected'].map(lambda s:'King' in s)].sort_values('Number of Customers Affected').tail(3)
# let's look at 1/19/2012 7am to 1/20/2012 3pm

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Month,Event Date and Time,Restoration Date and Time,Duration,Utility/Power Pool,NERC Region,Area Affected,Type of Disturbance,Loss (megawatts),Number of Customers Affected
70,2015,8.0,08/29/2015 11:00 AM,09/04/2015 3:00 PM,"148 Hours, 0 Minutes",Puget Sound Energy,WECC,"King County, Skagit County, Whatcom County, Ki...","Loss of electric service to more than 50,000 c...",250,250000.0
2,2012,1.0,01/19/2012 7:00 AM,01/20/2012 3:00 PM,"32 Hours, 0 Minutes",Puget Sound Energy,WECC,"King, Pierce and Thurston Counties, Washington",Severe Weather - Winter Storm,1600,426000.0
52,2016,8.0,08/24/2016 7:18 PM,08/24/2016 7:47 PM,"0 Hours, 29 Minutes",Peak Reliability,WECC,Washington: King County;,Electrical System Separation (Islanding) where...,9232,NaN


In [64]:
# scrape for tweets from that outage period for king county wa
tweets = query_tweets('outage OR blackout near:\"King County, WA\"', 10,
                      begindate=datetime.date(2012,1,18),
                      enddate=datetime.date(2012,1,21))

INFO: queries: ['outage OR blackout near:"Pierce County, WA" since:2012-01-18 until:2012-01-19', 'outage OR blackout near:"Pierce County, WA" since:2012-01-19 until:2012-01-20', 'outage OR blackout near:"Pierce County, WA" since:2012-01-20 until:2012-01-21']
INFO: Got 4 tweets (4 new).
INFO: Got 4 tweets (0 new).
INFO: Got 4 tweets (0 new).


In [65]:
for tweet in tweets:
    print(tweet.text)

When I'd the power outage over!?!?!?!
Power outage...
#wherearethecandles?
When I'd the power outage over!?!?!?!
Power outage...
#wherearethecandles?


State-only information is not especially useful to us, but it is not unusuable. All other information is counties; we will map by these. 

In [ ]:
# list of counties